In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

In [ ]:
import pyspark
from pyspark.sql import SQLContext
# Requirements for connection 
spark = pyspark.sql.SparkSession.builder \
   .master("local") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
# Spark context -  or connection 
sc = spark.sparkContext
# SQL context - or connection 
sqlContext = SQLContext(sc)

In [ ]:
data = sc.textFile("Taxi.csv")

In [ ]:
# Take the first n elements of the RDD
data.take(2)

In [ ]:
# Explore the columns names
data.first()

In [ ]:
data.top(1)

In [ ]:
type(data)

In [ ]:
# Schemas from SQL using a csv in pyspark
data2  = sqlContext.read.csv("taxi_dec.csv", header = True)

In [ ]:
type(data2)

In [ ]:
data2.printSchema()

In [ ]:
data2.dropna()

In [ ]:
data2.select("passenger_count").describe().show()

In [ ]:
data2.columns # Column Names
data2.count() # Row Count
len(data2.columns) # Column Count

In [ ]:
data2.select('passenger_count', 'trip_distance', 'total_amount').describe().show()

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
data2.select([count(when(isnan(c), c)).alias(c) for c in data2.columns]).show()

In [ ]:
data2.dropna().count()

In [ ]:
data = data2.filter((data2.passenger_count <= 4) & (data2.passenger_count > 0))
data = data.filter(data.total_amount >= 2.5)
data = data.filter((data.total_amount < 200) & (data.trip_distance < 100))

In [ ]:
data = data.filter(data.trip_distance > 0)

In [ ]:
data.select('passenger_count', 'total_amount').describe().show()

In [ ]:
data.select('tpep_pickup_datetime', 'tpep_dropoff_datetime').describe().show()

In [ ]:
pickups = data.select(col('PULocationID').alias('LocationID'))\
                        .groupBy('LocationID')\
                        .count()\
                        .sort('LocationID')\
                        .select(col('count').alias('pickups'), col('LocationID'))

In [ ]:
dropoffs = data.select(col('DOLocationID')\
                        .alias('LocationID'))\
                        .groupBy('LocationID')\
                        .count()\
                        .sort('LocationID')\
                        .select(col('count').alias('dropoffs'), col('LocationID'))

In [ ]:
picks_drops = pickups.join(dropoffs, on= ['LocationID'])

In [ ]:
picks_drops.show()

In [ ]:
picks_drops.show()

In [ ]:
picks_drops = picks_drops.toPandas()

In [ ]:
picks_drops.head()

In [ ]:
picks_drops['total'] = picks_drops['pickups'] + picks_drops['dropoffs']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
NYC_map = geopandas.read_file('taxi_zones/taxi_zones.shp')
NYC_map.crs

In [ ]:
NYC_map.tail()

In [ ]:
picks_drops['LocationID'] = picks_drops['LocationID'].astype(int)
picks_drops.dtypes

In [ ]:
NYC = NYC_map.merge(picks_drops, on = 'LocationID')


In [ ]:
NYC.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(1, 1)

In [ ]:
NYC.plot(column = 'total', cmap='OrRd', scheme='quantiles')

In [ ]:
top_pickups = NYC[['pickups', 'zone', 'borough']].sort_values('pickups', ascending = False)

In [ ]:
top_pickups.head(10)

In [ ]:
sc.close()